In [16]:
import pandas as pd
m = pd.read_csv("Airbnb_Texas_Rentals.csv",encoding = 'utf8')

In [17]:
#we found words like '\\n' in the dataset so we cleaned it
m = m.replace({r'\s+$': '', r'^\s+': ''}, regex=True).replace(r'\\n',  ' ', regex=True)
m = m.replace({r'\s+$': '', r'^\s+': ''}, regex=True).replace(r'\\t',  ' ', regex=True)
m = m.replace({r'\s+$': '', r'^\s+': ''}, regex=True).replace(r'\\r',  ' ', regex=True)

In [18]:
#create tsv files,we droped the first column that was a prroblem
from os.path import join as pjoin
import csv
m = m.drop(['Unnamed: 0'], axis=1)

In [19]:
#we create the vocabulary of preprocessed documents,but we don't modify the documents because we''l use them in search engine
import nltk
import csv
import re
import string
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
tokenizer = RegexpTokenizer(r'\w+|\$')
ps = SnowballStemmer('english')
dic = {}
for i in range(18259):
    with open(r'docu_hw3\doc_'+str(i)+'.tsv',encoding='utf8') as tsvfile:
         tsvreader = list(csv.reader(tsvfile, delimiter="\t"))
    l1 = tsvreader[0][4]
    l2 = tsvreader[0][7]
    l = l1+ ' ' +l2
    l = l.lower()
    l = tokenizer.tokenize(l)
    for r in l :
        if not r in stop_words:
            sr = ps.stem(r)
            if not  sr in dic:
                dic[sr] = [i]
            else:
                dic[sr]+=[i]
                


[nltk_data] Downloading package stopwords to C:\Users\Matteo
[nltk_data]     Cavalletti\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [20]:
# create dictionary of words
dic_word = {}
dic_assoc = {}
c = 0
for key in dic:
    dic_word[key] = c
    dic_assoc[c] = dic[key]
    c += 1

In [21]:
# create dictionary of documents
dic_doc={}
for i in range(18259):
    dic_doc[r'docu_hw3\doc_'+str(i)+'.tsv']= i

In [25]:
word = 'good peaceful mind'
word_list = word.split(' ')

list_doc_list = []

for w in word_list:
        curWord = ps.stem(w)
        
        doc_list = dic_assoc[dic_word[curWord]]
        list_doc_list.append(doc_list)  
        
list_intersect = list_doc_list[0]

def intersection(lst1, lst2): 
    lst3 = [value for value in lst1 if value in lst2] 
    return lst3 
        
for docList in list_doc_list:
    list_intersect = intersection(list_intersect, docList)

        
for doc in list_intersect:
    with open(r'docu_hw3\doc_'+ str(doc) + '.tsv',encoding='utf8') as tsvfile:
         tsvreader = list(csv.reader(tsvfile, delimiter="\t"))
    title = tsvreader[0][7]        
    description = tsvreader[0][4]
    city = tsvreader[0][2]
    url = tsvreader[0][8]
    
    print(title + '\t' + description + '\t' + city + '\t' + url + '\n')


Fully Renovated Vacation Home in North Plano	This beautiful home located in a nice and convenient community in North Plano with best schools around. You can stay here with a peaceful mind, because we have prepared everything brand new for you. You will find everything a hotel provide here, and much more! High speed wifi (ultimate fast speed up to 200 Mbps), 55 inch Smart TV(Netflix and local channels), double garage, outdoor grill, study area, etc. It is good for couples, business travellers, families (with kids), and big groups. BOOK NOW!	Plano	https://www.airbnb.com/rooms/15028039?location=Celina%2C%20TX

The Hannley Ranch	The adorable Hannley's Ranch House is on 6 acres of quiet and peaceful land, it is so close to Sea World, Fiesta Texas, La Cantera Shopping and many Restaurants . You’ll love this place because of the scenery and mind escaping atmosphere. The Hannley Ranch is good for couples, solo adventurers, business travelers, and families (with kids). Enjoy a dip in the Jacuzz